In [1]:
%reload_ext autoreload
%autoreload 2

import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()

import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 
from accelerate import Accelerator
import torch.nn.functional as F
# from einops import rearrange, pack, unpack
import sys
import os

os.environ["WANDB_DISABLED"] = "true"

#get cur dir
cur_dir = os.getcwd()
if 'miu' in cur_dir:
    sys.path.append('/home/miu/Projects/NLPLab/')
elif 'kiki' in cur_dir:
    sys.path.append('/home/kiki/dados_servidor/Servidor/NLPLab/')
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2.1.2


## PreTrain

In [2]:
model_training='hf'#'bert' , 'tokemonster
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#com retnet. Confirmado que é O(n). Se lembrando que não existe mta vantagem além do custo crescer sequencialmente, já que o conteudo 
#(4090) Para 84 tokens(tokenmonster salva 35%) batch size 200 e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias
#(4090) Para 168 tokens(tokenmonster salva 35%) batch size 100 e 16000 vocab (Electra): 1000 = 3.7s, 10000 = 37.5s,
#(4090) Para 1000 tokens(tokenmonster salva 35%) batch size 12 e 16000 vocab (Electra):            , 10000 = 4min32s,


#HIPOTÉTICO(Se conseguisse representar um texto com menos tokens):
#(4090) Para 24 tokens(tokenmonster salva 35%)(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias
#HIPOTÉTICO(Se diminuisse o vocabulario para 1024 tokens diferentes)
#(4090) Para 84 tokens(tokenmonster salva 35%) e 1024 vocab: 1000 =1.6s , 10000 = 16.2s,

#losses:
#1m dataset, electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#10m dataset, 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra) iteration:  49980 , total_loss:  20.8733185450236. 339min total


LIMIT_DATASET = 10000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
# VOCAB_SIZE = 32_768  # from Cramming
# DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
# MODEL_MAX_SEQ_LEN = 128  # from Cramming

VOCAB_SIZE = 32_768  # tokenmonster
# VOCAB_SIZE = 1_024  # tokenmonster
DEVICE_BATCH_SIZE = 100 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
MODEL_MAX_SEQ_LEN = 128 # token_monster


MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')

from utils.process_tokenizer import get_tokenizer
# tokenizer,tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'clm',MODEL_MAX_SEQ_LEN+2)
tokenized_dataset,norm,vocab,tokenizer=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE
                                                     ,TOKENIZER_PATH,'mlm',MODEL_MAX_SEQ_LEN+2,tok_type='hf'
                                                     ,tokenizer_name='bert-base-uncased')
# tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'smlm',MODEL_MAX_SEQ_LEN+2)
MASK_ID=vocab.tokenize("[MASK]")[0]
N_DOMAINS=3


batch_size:  2000


Resolving data files:   0%|          | 0/53 [00:00<?, ?it/s]

loaded dataset!!


In [ ]:
from x_transformers import TransformerWrapper, Decoder,Encoder

#encoder decoder
colt5_dict={
            'encoder':{'is_colt5_encoder':True,'light_ff_mult':0.5,'heavy_ff_mult':4,'num_heavy_tokens':30,
                   'light_dim_head':64,'light_heads':8,'light_window_size':128,
                   'heavy_dim_head':64,'heavy_heads':8,'num_heavy_tokens_q':30,
                   'num_heavy_tokens_kv':30},
            }

# model = TransformerWrapper(
model = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN,
    attn_layers = Encoder(
        dim = int(768),
        depth = 12,
        heads =12,
        attn_flash = True,
        colt5_dict=colt5_dict['encoder'],
        # rel_pos_bias = True
    ),
    emb_dim=128,
)

In [ ]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence

train_dataloader = DataLoader(
        tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
    )
from utils.utils import get_optimizer_scheduler
optimizer,lr_scheduler,max_train_steps = get_optimizer_scheduler(model,train_dataloader,gradient_accumulation_steps,learning_rate=5e-5,weight_decay=0, num_warmup_steps=0, max_train_steps=None,lr_scheduler_type='linear',num_train_epochs=1)

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

In [ ]:
# Only show the progress bar once on each machine.
import tqdm
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    # for step, batch in enumerate(train_dataloader):
    for step, batch in enumerate(tqdm.tqdm(train_dataloader)):

        # with accelerator.accumulate(model):
       
        logits=model(batch['input_ids']) 
        loss = F.cross_entropy(logits.transpose(1, 2),batch['input_ids'],ignore_index = PAD_ID)
        count_amostra+=int(len(batch['input_ids']))
        # print(loss)
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()       
        
        # if step%steps_log==0:
        #     print('iteration: ',step,', total_loss: ',total_loss/steps_log)
        #     print('count_amostra:',count_amostra)
        #     total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1

    if completed_steps >= max_train_steps:
        break

    model.eval()
    losses = []


  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [13:01:40<00:00,  2.13it/s] 


In [ ]:

#convert to huggingface Transformers

from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast

config = BertConfig(
    vocab_size=VOCAB_SIZE+8,
    max_position_embeddings=MODEL_MAX_SEQ_LEN,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=2,
)

bert_model=BertForMaskedLM(config=config)
bert_model.bert=model

In [ ]:
bert_model.save_pretrained('models/bert')

## Evaluate

### QA Trainer hf

In [3]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [4]:
max_length = MODEL_MAX_SEQ_LEN#384
stride = 100
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [5]:
validation_dataset = dataset["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["test"].column_names,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

thread '<unnamed>' panicked at /home/runner/work/tokenizers/tokenizers/tokenizers/src/tokenizer/encoding.rs:319:9:
`stride` must be strictly less than `max_len=100` (note that `max_len` may be shorter than the max length of the original model, as it subtracts the number of special characters
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
thread '<unnamed>' panicked at /home/runner/work/tokenizers/tokenizers/tokenizers/src/tokenizer/encoding.rs:319:9:
`stride` must be strictly less than `max_len=95` (note that `max_len` may be shorter than the max length of the original model, as it subtracts the number of special characters
thread '<unnamed>' panicked at /home/runner/work/tokenizers/tokenizers/tokenizers/src/tokenizer/encoding.rs:319:9:
`stride` must be strictly less than `max_len=98` (note that `max_len` may be shorter than the max length of the original model, as it subtracts the number of special characters
thread '<unnamed>' panicked at /home/runner/w

PanicException: `stride` must be strictly less than `max_len=98` (note that `max_len` may be shorter than the max length of the original model, as it subtracts the number of special characters

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=tokenizer.model_max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        # print('sequence_ids: ',sequence_ids)
        # print('sequence_ids: ',len(sequence_ids))

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1 and idx < len(sequence_ids) - 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
from transformers import DefaultDataCollator

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names,batch_size=4)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 1404.71 examples/s]


In [13]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

#BertForMaskedLM
from transformers import BertForMaskedLM

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)
# bert_model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path='bert-base-uncased')
# bert_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")
# bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert_spanbert")
if 'miu' in cur_dir:
    bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/miu/Projects/NLPLab/berts/models/bert")
else:
    bert_model = AutoModelForQuestionAnswering.from_pretrained("/home/kiki/dados_servidor/Servidor/NLPLab/berts/models/bert")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /home/miu/Projects/NLPLab/berts/models/bert and are newly initialized: ['bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.attention.output.dense.bias', 'bert.encoder.layer.1.attention.output.dense.weight', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.8.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.8.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.output.dense.weight', 'bert.encoder.layer.2.attention.self.query.weight', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.4.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.9.attention.self.value.bias', 'bert.encoder.layer.5.attention.self.query.weight', 'qa_outputs.bias', 'bert.encoder.layer.8.attention.sel

In [14]:
trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abcp4. Use `wandb login --relogin` to force relogin


 20%|██        | 500/2500 [00:39<02:38, 12.61it/s]

{'loss': 4.7581, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


 40%|████      | 1000/2500 [01:20<01:57, 12.82it/s]

{'loss': 4.0755, 'learning_rate': 1.2e-05, 'epoch': 2.0}


...

100%|██████████| 2500/2500 [03:21<00:00, 12.80it/s]

{'loss': 3.3212, 'learning_rate': 8e-09, 'epoch': 5.0}


100%|██████████| 2500/2500 [03:22<00:00, 12.37it/s]

{'train_runtime': 206.7869, 'train_samples_per_second': 96.718, 'train_steps_per_second': 12.09, 'train_loss': 3.8819138671875, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=3.8819138671875, metrics={'train_runtime': 206.7869, 'train_samples_per_second': 96.718, 'train_steps_per_second': 12.09, 'train_loss': 3.8819138671875, 'epoch': 5.0})

In [15]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate

metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [16]:
import evaluate

predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["test"])

100%|██████████| 1000/1000 [00:00<00:00, 1607.05it/s]


{'exact_match': 3.9, 'f1': 8.451318538500729}

100%|██████████| 1000/1000 [00:00<00:00, 1573.49it/s]


{'exact_match': 4.2, 'f1': 8.794485888565596}

In [ ]:
#100k 512 tokens
{'exact_match': 4.2, 'f1': 8.794485888565596}

## SEQ trainer

In [2]:
from transformers import AutoTokenizer, AutoModel,AutoConfig,AutoModelForSequenceClassification

# model_path ="bert-base-uncased"
# model_path ="/home/miu/Projects/crammed-bert"
model_path ="models/bert"

tokenizer_path="bert-base-uncased"
# tokenizer_path="/home/miu/Projects/crammed-bert"

# model=AutoModel.from_pretrained(model_path, return_dict=False)
config = AutoConfig.from_pretrained(model_path)
# classifier_model=AutoModelForSequenceClassification.from_pretrained(model_path, config=config)
# classifier_model.vocab_size = 32_768
# classifier_model.vocab_size = 30_000
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.model_max_length=128
MODEL_MAX_SEQ_LEN=tokenizer.model_max_length
HF_MODEL=True
MODEL_MAX_SEQ_LEN

128

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [4]:
task = "mrpc"
model_checkpoint = model_path
batch_size = 16

In [5]:
from datasets import load_dataset, load_metric

In [6]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

/tmp/ipykernel_324388/1389288479.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)
/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [7]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys[task]


In [8]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [9]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models/bert and are newly initialized: ['bert.encoder.layer.9.attention.self.value.bias', 'classifier.bias', 'bert.encoder.layer.5.output.dense.bias', 'bert.encoder.layer.7.output.dense.weight', 'bert.encoder.layer.8.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.5.attention.self.query.bias', 'bert.encoder.layer.10.output.dense.bias', 'bert.encoder.layer.3.attention.output.dense.weight', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.10.attention.self.key.weight', 'bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.3.attention.self.key.bias', 'classifier.weight', 'bert.encoder.layer.3.output.dense.weight', 'bert.encoder.layer.5.attention.self.key.weight', 'bert.encoder.layer.2.attention.self.key.weight', 'bert.embeddings.LayerNorm.bias', 'bert.

#### Custom

In [17]:
encoded_dataset = encoded_dataset.remove_columns(["sentence1", "sentence2", "idx"])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch")
encoded_dataset["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [18]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    encoded_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    encoded_dataset["validation"], batch_size=8, collate_fn=data_collator
)

In [19]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()
optimizer = AdamW(model.parameters(), lr=3e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


In [22]:

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    # for batch in train_dataloader:
    for batch in tqdm(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/459 [00:00<?, ?it/s]

100%|██████████| 459/459 [05:00<00:00,  1.53it/s]


In [23]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()

for batch in tqdm(eval_dataloader):
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

100%|██████████| 51/51 [00:08<00:00,  6.13it/s]


{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

#### Trainer

In [11]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "no",
    # learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    learning_rate=4e-5,
    warmup_steps=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    # load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    report_to="none",
)

In [12]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [13]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.623602,0.683824,0.812227
2,No log,0.626583,0.683824,0.812227
3,0.646300,0.617421,0.678922,0.786297
4,0.646300,0.633018,0.676471,0.791798
5,0.609000,0.629557,0.681373,0.785479


TrainOutput(global_step=1150, training_loss=0.6179507247261379, metrics={'train_runtime': 45.9213, 'train_samples_per_second': 399.379, 'train_steps_per_second': 25.043, 'total_flos': 716101959374880.0, 'train_loss': 0.6179507247261379, 'epoch': 5.0})

In [14]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.423086,0.828431,0.879725
2,No log,0.405031,0.838235,0.886207
3,0.437900,0.495247,0.848039,0.894198
4,0.437900,0.608123,0.860294,0.901893
5,0.144300,0.650582,0.865196,0.904679


TrainOutput(global_step=1150, training_loss=0.2635675811767578, metrics={'train_runtime': 45.7017, 'train_samples_per_second': 401.298, 'train_steps_per_second': 25.163, 'total_flos': 716101959374880.0, 'train_loss': 0.2635675811767578, 'epoch': 5.0})

In [31]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.613799,0.683824,0.812227
2,No log,0.593775,0.686275,0.799373
3,0.627900,0.622357,0.713235,0.807249
4,0.627900,0.688114,0.644608,0.725898
5,0.496900,0.775074,0.678922,0.771379


TrainOutput(global_step=1150, training_loss=0.5382972617771314, metrics={'train_runtime': 45.7071, 'train_samples_per_second': 401.251, 'train_steps_per_second': 25.16, 'total_flos': 716101959374880.0, 'train_loss': 0.5382972617771314, 'epoch': 5.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.6295565366744995,
 'eval_accuracy': 0.6813725490196079,
 'eval_f1': 0.7854785478547854,
 'eval_runtime': 0.2419,
 'eval_samples_per_second': 1686.844,
 'eval_steps_per_second': 107.495,
 'epoch': 5.0}

In [32]:
trainer.evaluate()

{'eval_loss': 0.7750740647315979,
 'eval_accuracy': 0.678921568627451,
 'eval_f1': 0.7713787085514833,
 'eval_runtime': 0.2396,
 'eval_samples_per_second': 1703.04,
 'eval_steps_per_second': 108.527,
 'epoch': 5.0}

In [ ]:
trainer.evaluate()

  0%|          | 0/26 [00:00<?, ?it/s]

{'eval_loss': 0.5459453463554382,
 'eval_accuracy': 0.8578431372549019,
 'eval_f1': 0.8993055555555555,
 'eval_runtime': 0.4866,
 'eval_samples_per_second': 838.425,
 'eval_steps_per_second': 53.429,
 'epoch': 5.0}

## SEQ custom

In [2]:
from transformers import AutoTokenizer, AutoModel,AutoConfig,AutoModelForSequenceClassification

# model_path ="bert-base-uncased"
# model_path ="/home/miu/Projects/crammed-bert"
model_path ="models/bert"

tokenizer_path="bert-base-uncased"
# tokenizer_path="/home/miu/Projects/crammed-bert"

model=AutoModel.from_pretrained(model_path, return_dict=False)
config = AutoConfig.from_pretrained(model_path)
# classifier_model=AutoModelForSequenceClassification.from_pretrained(model_path, config=config)
# classifier_model.vocab_size = 32_768
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.model_max_length=128
MODEL_MAX_SEQ_LEN=tokenizer.model_max_length
HF_MODEL=True
MODEL_MAX_SEQ_LEN

Some weights of BertModel were not initialized from the model checkpoint at models/bert and are newly initialized: ['bert.encoder.layer.1.attention.self.key.weight', 'bert.encoder.layer.9.output.dense.bias', 'bert.encoder.layer.5.attention.self.value.bias', 'bert.encoder.layer.5.output.dense.weight', 'bert.encoder.layer.11.output.LayerNorm.bias', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.1.output.dense.weight', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.5.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.intermediate.dense.bias', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.4.attention.output.dense.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.bias', 'bert.encoder.layer.7.intermediate.dense.bias', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.3.attention.self.value.bias', 'bert.encoder.layer.8.attention.output

128

In [3]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x,att_mask=None):
        if self.layer == -1:
            return self.net(x,att_mask)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x,att_mask)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [4]:
# model_extractor = HiddenLayerExtractor(model,-2)
model_extractor = HiddenLayerExtractor(model,-1)

In [5]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.encoder = model_extractor
        
        # self.d = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.l1 = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN), 768)
        self.l2 = torch.nn.Linear(768, 128)
        self.l3 = torch.nn.Linear(128, 2)
        self.t=torch.nn.Tanh()
    
    def forward(self, ids,att_mask=None):
        
        output= self.encoder(ids,att_mask=att_mask)[0]
        #get cls token
        # output=output[:,0]
        output = self.fl(output)
        output = self.t(output)
        output = self.l1(output)
        output = self.l2(output)
        output = self.l3(output)
            
        return output

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')

In [6]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.ls=[]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        
        #with hf tokenizer
        s=self.dataset["sentence1"][i]+'[SEP]'+self.dataset["sentence2"][i]
        t = tokenizer(s, padding='max_length', truncation=True, max_length=MODEL_MAX_SEQ_LEN, return_tensors="pt")
        tokens=t['input_ids']
        att_mask=t['attention_mask']
        
        #Tokenizer do crammed bert ta extraindo ids fora no vocabulario total!!!
        for k in range(len(tokens)):
            for l in range(len(tokens[k])):
                if tokens[k][l]>=tokenizer.vocab_size-2500:
                    tokens[k][l]=0
        
        self.ls.append(i)

        label=self.dataset["label"][i]
        
        d={'input_ids':tokens,'label':label,'attention_mask':att_mask}
        return d

from torch.utils.data import DataLoader

epochs=4
batch_size=16
task='mrpc'
dataset = load_dataset('glue', task, split='train')

tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=batch_size
    )

def group_parameters(model, limited_decay_keys,weight_decay):
    model_parameters = list(model.named_parameters())
    if len(limited_decay_keys) > 0:
        grouped_parameters = optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model_parameters if not any(nd in n for nd in limited_decay_keys)],
                "weight_decay": weight_decay,
            },
            {
                "params": [p for n, p in model_parameters if any(nd in n for nd in limited_decay_keys)],
                "weight_decay": 0.0,
            },
        ]
    else:
        grouped_parameters = [p for n, p in model_parameters]
    return grouped_parameters


In [7]:
#hf bert
# optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=2e-5,weight_decay=0.01)

# #cramming bert
from transformers import get_cosine_schedule_with_warmup
#'AdamW', 'lr': 4e-05, 'betas': [0.9, 0.98], 'eps': 1e-06, 'weight_decay': 0.0, 'amsgrad': False
# optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=4e-5)
grouped_parameters = group_parameters(classifier_model, ['bias', 'LayerNorm.bias','LayerNorm.weight', 'norm'],0.01)
# optimizer = torch.optim.AdamW(params =  classifier_model.parameters(), lr=4e-5,weight_decay=0.0,amsgrad=False,betas=(0.9, 0.98),eps=1e-06)
optimizer = torch.optim.AdamW(params =  grouped_parameters, lr=4e-05,weight_decay=0.0,amsgrad=False,betas=(0.9, 0.98),eps=1e-06)
num_cycles=0.5
num_training_steps=len(train_dataloader)*batch_size*epochs
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.1, num_cycles=num_cycles, num_training_steps=num_training_steps)

loss_list = []


In [8]:
#FINETUNING
import tqdm
import numpy as np
from torch.autograd import Variable

classifier_model.train()
loss_fct = torch.nn.CrossEntropyLoss()
running_loss=0

scaler = torch.cuda.amp.GradScaler(enabled=True)
for i in range(epochs):
    predictions=[]
    labels=[]
    for _,data in enumerate(tqdm.tqdm(train_dataloader)):
        
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        att_mask = data['attention_mask'].to('cuda', dtype = torch.long)
        if len(ids.shape)==3:
            ids=ids.squeeze(1)
        targets = data['label'].to('cuda', dtype = torch.long)

        outputs = classifier_model(ids,att_mask)
        # outputs=outputs[0]
        
        loss = loss_fct(outputs.view(-1, 2), targets.view(-1))
        # loss=outputs.loss

        running_loss+=loss.item()
        
        scaler.scale(loss).backward()
        # if _%2==0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()

        outputs=torch.argmax(outputs, dim=1).cpu()
        targets=targets.cpu()
        
        outputs=[int(o.item()) for o in outputs]
        targets=[int(o.item()) for o in targets]

        predictions+=outputs
        labels+=targets
    print('epoch: ',i,' loss: ',running_loss/len(train_dataloader))
    from sklearn.metrics import classification_report
    print(classification_report(labels, predictions))

  0%|          | 0/230 [00:00<?, ?it/s]

100%|██████████| 230/230 [00:38<00:00,  5.94it/s]


epoch:  0  loss:  0.7191867249167484
              precision    recall  f1-score   support

           0       0.31      0.09      0.14      1194
           1       0.67      0.90      0.77      2474

    accuracy                           0.64      3668
   macro avg       0.49      0.50      0.46      3668
weighted avg       0.56      0.64      0.57      3668



100%|██████████| 230/230 [00:39<00:00,  5.78it/s]


epoch:  1  loss:  1.3523259504981664
              precision    recall  f1-score   support

           0       0.53      0.03      0.06      1194
           1       0.68      0.99      0.80      2474

    accuracy                           0.68      3668
   macro avg       0.60      0.51      0.43      3668
weighted avg       0.63      0.68      0.56      3668



100%|██████████| 230/230 [00:39<00:00,  5.89it/s]


epoch:  2  loss:  1.9420064615814583
              precision    recall  f1-score   support

           0       0.60      0.20      0.30      1194
           1       0.71      0.94      0.81      2474

    accuracy                           0.70      3668
   macro avg       0.66      0.57      0.56      3668
weighted avg       0.67      0.70      0.64      3668



100%|██████████| 230/230 [00:39<00:00,  5.77it/s]


epoch:  3  loss:  2.415924946283517
              precision    recall  f1-score   support

           0       0.70      0.49      0.58      1194
           1       0.78      0.90      0.84      2474

    accuracy                           0.77      3668
   macro avg       0.74      0.69      0.71      3668
weighted avg       0.76      0.77      0.75      3668



In [9]:
#EVALUATE
# dataset = load_dataset('glue', task, split='validation')
dataset = load_dataset('glue', task, split='test')
tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=8
    )

In [10]:
classifier_model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    if len(ids.shape)==3:
        ids=ids.squeeze(1)
    targets = data['label'].to('cuda', dtype = torch.long)
    att_mask = data['attention_mask'].to('cuda', dtype = torch.long)
    
    outputs = classifier_model(ids,att_mask)
    # outputs=outputs[0]
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets


In [11]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.46      0.35      0.40       578
           1       0.71      0.79      0.75      1147

    accuracy                           0.65      1725
   macro avg       0.58      0.57      0.57      1725
weighted avg       0.63      0.65      0.63      1725



              precision    recall  f1-score   support

           0       0.40      0.78      0.53       578
           1       0.78      0.40      0.53      1147

    accuracy                           0.53      1725
   macro avg       0.59      0.59      0.53      1725
weighted avg       0.65      0.53      0.53      1725



In [ ]:
#Bert 3 layers  10000 dataset
#               precision    recall  f1-score   support

#            0       0.61      0.09      0.15       129
#            1       0.70      0.97      0.81       279

#     accuracy                           0.69       408
#    macro avg       0.65      0.53      0.48       408
# weighted avg       0.67      0.69      0.60       408

#bert normal
#               precision    recall  f1-score   support

#            0       0.67      0.90      0.77       129
#            1       0.94      0.80      0.86       279

#     accuracy                           0.83       408
#    macro avg       0.81      0.85      0.82       408
# weighted avg       0.86      0.83      0.83       408


## DensePhrases

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [3]:
#load model and tokenizer
from transformers import AutoTokenizer, AutoModel,AutoModelForQuestionAnswering,AutoConfig
from transformers import TrainingArguments
from transformers.utils import cached_file

# tokenizer_path="bert-base-uncased"
# model_path ="/home/miu/Projects/NLPLab/berts/models/bert"
# model_path="bert-base-uncased"

# tokenizer_path="/home/miu/Projects/crammed-bert"
# model_path ="/home/miu/Projects/crammed-bert"

model_path='/home/miu/Projects/NLP/spanbert'
model_path='princeton-nlp/densephrases-multi'
tokenizer_path="bert-base-cased"


tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.model_max_length=128


#model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

config = AutoConfig.from_pretrained("bert-base-uncased")
# config = AutoConfig.from_pretrained("/home/miu/Projects/crammed-bert")

In [4]:
from transformers import PreTrainedModel
import copy
from torch.nn import CrossEntropyLoss
# Encoders: three LMs

class Encoder(PreTrainedModel):
# class Encoder():
    def __init__(self,return_phrase=False,
                 return_query=False):
        super().__init__(config)
        # super().__init__()
        self.phrase_encoder = AutoModel.from_pretrained(model_path)
        self.query_start_encoder = copy.deepcopy(self.phrase_encoder)
        self.query_end_encoder = copy.deepcopy(self.phrase_encoder)
        # Additional parameters
        self.filter_linear = torch.nn.Linear(768, 2)
        self.return_phrase = return_phrase
        self.return_query = return_query

    def embed_phrase(self, input_ids, attention_mask, token_type_ids):
        
        """ Get phrase embeddings (token-wise) """
        outputs = self.phrase_encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return outputs[0], outputs[0]

    def embed_query(self, input_ids_, attention_mask_, token_type_ids_):
        """ Get query start/end embeddings """
        # print('input_ids_.shape: ',input_ids_.shape)
        # Two LM based query reps
        outputs_s_ = self.query_start_encoder(
            input_ids_,
            attention_mask=attention_mask_,
            token_type_ids=token_type_ids_,
        )
        outputs_e_ = self.query_end_encoder(
            input_ids_,
            attention_mask=attention_mask_,
            token_type_ids=token_type_ids_,
        )
        sequence_output_s_ = outputs_s_[0]
        sequence_output_e_ = outputs_e_[0]
        query_start = sequence_output_s_[:,:1,:]
        query_end = sequence_output_e_[:,:1,:]
        return query_start, query_end
    
    def forward(self,
            input_ids=None, attention_mask=None, token_type_ids=None,
            input_ids_=None, attention_mask_=None, token_type_ids_=None,
            start_positions=None, end_positions=None,
            neg_input_ids=None, neg_attention_mask=None, neg_token_type_ids=None,
            example_id=None,):
        # Context-side
        if input_ids is not None:
            assert len(input_ids.size()) == 2
            start, end = self.embed_phrase(input_ids, attention_mask, token_type_ids)

            if neg_input_ids is not None:
                neg_start, neg_end = self.embed_phrase(neg_input_ids, neg_attention_mask, neg_token_type_ids)

            # Get filter logits
            filter_output = start[:]
            filter_start_logits, filter_end_logits = self.filter_linear(filter_output).chunk(2, dim=2)
            filter_start_logits = filter_start_logits.squeeze(2)
            filter_end_logits = filter_end_logits.squeeze(2)

            if self.return_phrase:
                return (start, end, filter_start_logits, filter_end_logits)

        # Query-side
        if input_ids_ is not None:
            assert len(input_ids_.size()) == 2
            query_start, query_end = self.embed_query(input_ids_, attention_mask_, token_type_ids_)

            if self.return_query:
                return (query_start, query_end)
        
        # Get dense logits
        start_logits = start.matmul(query_start.transpose(1, 2)).squeeze(-1)
        end_logits = end.matmul(query_end.transpose(1, 2)).squeeze(-1)
        dense_logits = start_logits.unsqueeze(2) + end_logits.unsqueeze(1)

        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)

            ignored_index = start_logits.size(1)
            start_positions.clamp_(0, ignored_index)
            end_positions.clamp_(0, ignored_index)

            # 1) Single-passage loss
            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(dense_logits.mean(2), start_positions)
            end_loss = loss_fct(dense_logits.mean(1), end_positions)
            single_loss = (start_loss + end_loss) / 2
            total_loss = single_loss

            return (total_loss,)

In [5]:
model=Encoder()

Some weights of BertModel were not initialized from the model checkpoint at princeton-nlp/densephrases-multi and are newly initialized: ['encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.2.intermediate.dense.weight', 'pooler.dense.bias', 'encoder.layer.5.attention.self.key.weight', 'encoder.layer.10.attention.self.key.bias', 'encoder.layer.5.attention.self.value.bias', 'encoder.layer.9.attention.self.key.weight', 'encoder.layer.4.output.dense.weight', 'encoder.layer.2.attention.output.dense.bias', 'encoder.layer.7.attention.self.key.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.9.attention.output.LayerNorm.bias', 'encoder.layer.7.attention.self.query.bias', 'encoder.layer.11.attention.self.query.weight', 'encoder.layer.1.attention.self.value.bias', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.11.attention.self.key.weight', 'encoder.layer.4.attention.output.LayerNorm.bias', 'encoder.layer.11.output.LayerNorm.bias', 'encoder.layer.1

In [6]:
# Preprocessing the datasets.
# Preprocessing is slighlty different for training and evaluation.
column_names = dataset["train"].column_names

question_column_name = "question" if "question" in column_names else column_names[0]
context_column_name = "context" if "context" in column_names else column_names[1]
answer_column_name = "answers" if "answers" in column_names else column_names[2]

# Padding side determines if we do (question|context) or (context|question).
pad_on_right = tokenizer.padding_side == "right"

# max_seq_length = tokenizer.model_max_length
max_seq_length = 128
max_query_length = 25#50
pad_on_right,max_seq_length

(True, 128)

In [7]:
# Training preprocessing
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    
    tokenized_examples = tokenizer(
        examples[context_column_name],
        truncation="only_first",
        max_length=max_seq_length,
        # stride=128,
        stride=64,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    tokenized_questions = tokenizer(
        examples[question_column_name],
        truncation="only_first",
        max_length=max_query_length,
        return_overflowing_tokens=False,
        return_offsets_mapping=False,
        padding="max_length" 
    )

    #bugfix: vocab tem 32000 tokens, mas embedding so tem 30522. Ajeitar!!!
    # #if any input ids is bigger than tokenizer.vocab_size set it to 0
    # for i in range(len(tokenized_questions["input_ids"])):
    #     for j in range(len(tokenized_questions["input_ids"][i])):
    #         if tokenized_questions["input_ids"][i][j]>=tokenizer.vocab_size-3000:
    #             tokenized_questions["input_ids"][i][j]=0
    # #if any input ids is bigger than tokenizer.vocab_size set it to 0
    # for i in range(len(tokenized_examples["input_ids"])):
    #     for j in range(len(tokenized_examples["input_ids"][i])):
    #         if tokenized_examples["input_ids"][i][j]>=tokenizer.vocab_size-3000:
    #             tokenized_examples["input_ids"][i][j]=0

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    

    # Inflate questions based on sample_mapping
    tokenized_examples['input_ids_'] = [tokenized_questions['input_ids'][i] for i in sample_mapping]
    tokenized_examples['token_type_ids_'] = [tokenized_questions['token_type_ids'][i] for i in sample_mapping]
    tokenized_examples['attention_mask_'] = [tokenized_questions['attention_mask'][i] for i in sample_mapping]

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        # cls_token_id = tokenizer.cls_token_id
        # cls_index = input_ids.index(cls_token_id)
        cls_index=0

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        #context_index = 1 if pad_on_right and args.append_title else 0
        context_index =  0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != context_index:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != context_index:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)



    #bugfix: limit to 1000. Ajeitar!!!
    
    tokenized_examples["input_ids"]=tokenized_examples["input_ids"][:1000]
    tokenized_examples["input_ids_"]=tokenized_examples["input_ids_"][:1000]
    tokenized_examples["attention_mask"]=tokenized_examples["attention_mask"][:1000]
    tokenized_examples["attention_mask_"]=tokenized_examples["attention_mask_"][:1000]
    tokenized_examples["token_type_ids"]=tokenized_examples["token_type_ids"][:1000]
    tokenized_examples["token_type_ids_"]=tokenized_examples["token_type_ids_"][:1000]
    tokenized_examples["start_positions"]=tokenized_examples["start_positions"][:1000]
    tokenized_examples["end_positions"]=tokenized_examples["end_positions"][:1000]
    return tokenized_examples

In [8]:
train_dataset = dataset["train"]
train_dataset = train_dataset.map(
                prepare_train_features,
                batched=True,
                num_proc=1,
                #remove_columns=column_names,
                #load_from_cache_file=not args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

Running tokenizer on train dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [9]:
from transformers import default_data_collator
from transformers import DataCollatorWithPadding,EvalPrediction
from datasets import load_metric
from utils.utils_qa import postprocess_qa_predictions

# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval", filter_threshold=-1e5):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions, save_rate = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=args.version_2_with_negative,
        n_best_size=args.n_best_size,
        max_answer_length=args.max_answer_length,
        null_score_diff_threshold=args.null_score_diff_threshold,
        output_dir=args.output_dir,
        log_level=0,
        prefix=stage,
        filter_threshold=filter_threshold,
    )
    # Format the result to the format the metric expects.
    if args.version_2_with_negative:
        formatted_predictions = [
            {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        ]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

    references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references), save_rate

In [10]:
from utils.trainer_qa import QuestionAnsweringTrainer
os.environ["WANDB_DISABLED"] = "true"

pad_to_max_length=True
metric = load_metric("squad")
# Data collator
# We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data
# collator.
data_collator = (
    default_data_collator
    # if args.pad_to_max_length
    # else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
    if pad_to_max_length
    else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
)

def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="no",
    # use_cpu=True,
    # learning_rate=2e-5,
    # learning_rate=4e-5,
    learning_rate=3e-5,
    num_train_epochs=5,
    warmup_steps=0.1,
    # lr_scheduler_type="cosine",
    lr_scheduler_type="linear",
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

# Initialize our Trainer
trainer = QuestionAnsweringTrainer(
    model=model,
    args=args,
    # args=None,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_380849/1562132463.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report

In [11]:
train_result = trainer.train(resume_from_checkpoint=False)

Step,Training Loss
500,7.408100
1000,4.348500
1500,3.686800
2000,3.170000
2500,2.786900


In [ ]:
#cramming original
500	7.016400
1000	4.341700
1500	3.777300
2000	3.374300
2500	3.108100

#dataset treinado 10m
# {'loss': 6.9726, 'learning_rate': 1.6016e-05, 'epoch': 1.0}
# {'loss': 4.4086, 'learning_rate': 1.2016000000000002e-05, 'epoch': 2.0}
# {'loss': 3.7937, 'learning_rate': 8.016e-06, 'epoch': 3.0}
# {'loss': 3.3883, 'learning_rate': 4.016e-06, 'epoch': 4.0}
# {'loss': 3.1216, 'learning_rate': 1.6e-08, 'epoch': 5.0}
# {'train_runtime': 189.2476, 'train_samples_per_second': 105.682, 'train_steps_per_second': 13.21, 'train_loss': 4.336957421875, 'epoch': 5.0}

#bert original
#  20%|██        | 501/2500 [01:03<04:10,  7.97it/s]
# {'loss': 4.8419, 'learning_rate': 1.6064e-05, 'epoch': 1.0}
#  40%|████      | 1001/2500 [02:05<03:07,  7.99it/s]
# {'loss': 0.999, 'learning_rate': 1.2064e-05, 'epoch': 2.0}
#  60%|██████    | 1501/2500 [03:08<02:05,  7.96it/s]
# {'loss': 0.4677, 'learning_rate': 8.064000000000001e-06, 'epoch': 3.0}
#  80%|████████  | 2001/2500 [04:08<01:00,  8.26it/s]
# {'loss': 0.2472, 'learning_rate': 4.064e-06, 'epoch': 4.0}
# 100%|██████████| 2500/2500 [05:08<00:00,  8.11it/s]
# {'loss': 0.1275, 'learning_rate': 6.4e-08, 'epoch': 5.0}
# {'train_runtime': 308.1526, 'train_samples_per_second': 64.903, 'train_steps_per_second': 8.113, 'train_loss': 1.3366608764648438, 'epoch': 5.0}


# 500	9.653200
# 1000	5.496600
# 1500	4.896600
# 2000	4.454100
# 2500	4.119600

### Inference

#### Create phrase vectors


In [12]:
from utils.utils_dp import convert_squad_to_hf
import logging

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

data_files = {}
data_files["test"] = convert_squad_to_hf('/home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000')

Saving 18829 (context, question, answer) triples.
Writing to /home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000_hf.json



In [13]:
data_files["test"]

'/home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000_hf.json'

In [14]:
extension = data_files["test"].split(".")[-1]
raw_datasets = load_dataset(extension, data_files=data_files, field="data")

column_names = raw_datasets["test"].column_names
context_column_name = "context" if "context" in column_names else column_names[1]

# Padding side determines if we do (question|context) or (context|question).
pad_on_right = tokenizer.padding_side == "right"
# max_seq_length = 512
max_seq_length = 128

Generating test split: 0 examples [00:00, ? examples/s]

In [15]:
raw_datasets['test'][0]

{'doc_idx': 0,
 'par_idx': 0,
 'title': 'Percy Fawcett',
 'context': ' Lieutenant Colonel Percy Harrison Fawcett (18 August 1867during or after 1925) was a British geographer, artillery officer, cartographer, archaeologist, and explorer of South America. Fawcett disappeared in 1925 (along with his eldest son, Jack, and one of Jack\'s friends, Raleigh Rimmell) during an expedition to find "Z"—his name for an ancient lost city which he and others believed existed in the jungles of Brazil. Percy Fawcett was born on 18 August 1867 in Torquay, Devon, England, to Edward Boyd Fawcett and Myra Elizabeth (née MacDougall). Fawcett received his early education at Newton Abbot Proprietary College along with Bertram Fletcher Robinson. Fawcett\'s India-born father was a Fellow of the Royal Geographical Society (RGS); his elder brother Edward Douglas Fawcett (1866–1960) was a mountain climber, an Eastern occultist, and the author of philosophical books and popular adventure novels.'}

In [16]:
offset = 0

# Validation preprocessing
def prepare_validation_features(examples, indexes):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[context_column_name],
        truncation="only_first",
        max_length=max_seq_length,
        # stride=128,
        stride=64,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" ,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    
    # Inflate doc_idxs based on sample_mapping
    tokenized_examples['doc_idx'] = [offset + examples['doc_idx'][i] for i in sample_mapping]

    # This example_id indicates the index of an original paragraph (not question id)
    tokenized_examples['example_id'] = [indexes[i] for i in sample_mapping]

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [17]:
examples = raw_datasets["test"]
accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps)

# Predict Feature Creation
with accelerator.main_process_first():
    dataset = examples.map(
        prepare_validation_features,
        batched=True,
        num_proc=1,
        with_indices=True,
        remove_columns=column_names,
        # load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on prediction dataset",
    )

# Data collator
# We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data
# collator.
data_collator = (
    default_data_collator
    )

Running tokenizer on prediction dataset:   0%|          | 0/18829 [00:00<?, ? examples/s]

In [18]:
per_device_train_batch_size=8

predict_dataset_for_model = dataset.remove_columns([ "offset_mapping","doc_idx","overflow_to_sample_mapping"])
predict_dataloader = DataLoader(
    predict_dataset_for_model, shuffle=True, collate_fn=data_collator, batch_size=per_device_train_batch_size,
)

model, predict_dataloader = accelerator.prepare(model, predict_dataloader)

In [19]:
len(predict_dataloader)

5882

In [20]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()
import tqdm
c=0
feature_id=0
all_predictions=[]
model.return_phrase=True
# for step, inputs in enumerate(predict_dataloader):
for inputs in tqdm.tqdm(predict_dataloader):
    outputs=model(**inputs)

    for item_idx, (start, end, filter_start, filter_end) in enumerate(zip(*outputs)):
        a= {'example_id': inputs['example_id'][item_idx].item(),
            'feature_id': 0, # dataloader should not be shuffled
            'start': to_numpy(start),
            'end': to_numpy(end),
            'filter_start': to_numpy(filter_start),
            'filter_end': to_numpy(filter_end),
            'inputs': inputs['input_ids'][item_idx].tolist(),
        }
        all_predictions.append(a)
    feature_id+=1
    c+=1
    if c>1000:
        break
outputs[0].shape,outputs[1].shape,outputs[2].shape,outputs[3].shape

  0%|          | 0/5882 [00:00<?, ?it/s]

 17%|█▋        | 1000/5882 [00:10<00:51, 95.64it/s]


(torch.Size([8, 128, 768]),
 torch.Size([8, 128, 768]),
 torch.Size([8, 128]),
 torch.Size([8, 128]))

In [21]:
len(all_predictions),all_predictions[0]['start'].shape

(8008, (128, 768))

#### Search Engine

In [22]:
inputs['input_ids'].shape

torch.Size([8, 128])

In [23]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] hours ; the last 100 mg / kg are infused over the remaining 16 hours of the protocol. Intravenous acetylcysteine has the advantage of shortening hospital stay, increasing both doctor and patient convenience, and allowing administration of activated charcoal to reduce absorption of both the paracetamol and any co - ingested drugs without concerns about interference with oral acetylcysteine. Intravenous dosing varies with weight, specifically in children. For patients less than 20 kg, the loading dose is 150 mg / kg in 3 mL / kg diluent, administered over 60 minutes ; [SEP]'

In [24]:
df=[]
query = 'tell me about the historical spending on defense'

for i in range(1):
    df.append({'doc_idx':0, 'par_idx':0, 'title':'', 'context':query})
p='/home/miu/Projects/NLPLab/DensePhrasesMinimal/query.json'
with open(p, 'w') as f:
        json.dump({'data': df}, f)
r_d = load_dataset(extension, data_files=p, field="data")
r_d=r_d['train']
# query_inputs = prepare_features(r_d['train'])
with accelerator.main_process_first():
    query_dataset = r_d.map(
        prepare_validation_features,
        batched=True,
        num_proc=1,
        with_indices=True,
        desc="Running tokenizer on prediction dataset",
    )
query_dataset = query_dataset.remove_columns(["par_idx", "offset_mapping","doc_idx","overflow_to_sample_mapping"])
query_dataloader = DataLoader(
    query_dataset, shuffle=True, collate_fn=data_collator, batch_size=1,
)
query_dataloader = accelerator.prepare(query_dataloader)

Generating train split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

In [25]:
model.return_query=True
model.return_phrase=False

for step, query_inputs in enumerate(query_dataloader):
    #print(query_inputs)
    query_outputs=model(input_ids_=query_inputs['input_ids'],attention_mask_=query_inputs['attention_mask'])
    
    break

In [26]:
import numpy as np
def search_mips(phrases_embs,query_emb):
    print(phrases_embs.shape,query_emb.shape)
    scores=[]
    for i in range(phrases_embs.shape[0]):
        batch=[]
        for j in range(phrases_embs.shape[1]):
            # if j%10==0:
                #batch.append(phrases_embs[i][j])
            s=np.dot(phrases_embs[i][j],query_emb)
            scores.append((i,j,s))

        #reduced_embs.append(batch)
    scores=np.asarray(scores)
    return scores

In [27]:
q_start=query_outputs[0][0].cpu().detach().numpy()[0]
q_end=query_outputs[1][0].cpu().detach().numpy()[0]
print(q_start.shape,q_end.shape)

(768,) (768,)


In [28]:
# scores=search_mips(outputs[0].cpu().detach().numpy(),q_start)
search_embs=np.asarray([all_predictions[i]['start'] for i in range(len(all_predictions))])
# search_ids = np.asarray([all_predictions[i]['inputs'] for i in range(len(all_predictions))])
print(search_embs.shape)
scores=search_mips(search_embs,q_start)

(8008, 128, 768)
(8008, 128, 768) (768,)


In [29]:
best_score = np.argmax(scores[:,2])
i=int(scores[best_score][0])
j=int(scores[best_score][1])
best_score,scores[best_score]

(986880, array([7710.        ,    0.        ,  651.54821777]))

In [30]:
# tokenizer.decode(inputs['input_ids'][i][j:])

In [31]:
inp=all_predictions[i]['inputs']
#decode
tokenizer.decode(inp[j:])

"[CLS] that he could inherit a fortune by murdering his uncle and cousin. Stapleton had taken Sir Henry's old boot because the new, unworn boot lacked his scent. The hound had pursued Selden to his death because of the scent on Sir Henry's old clothes. Mrs Stapleton had disavowed her husband's plot, so he had imprisoned her to prevent her from interfering. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

'2016 and 2017 defense spending budgets that " [ reflect ] the priorities necessary for our force today and in the future to best serve and protect our nation in a rapidly changing security environment. " in february 2018, the pentagon requested $ 686 billion for fy 2019. the john s. mccain national defense authorization act authorized department of defense appropriations for 2019 and established policies, but it did not contain the budget itself. on july 26, this bill passed in the house of representatives by 359 - 54. on august 1, the us senate passed it by 87 - 10. the bill was presented to president trump two days later. he signed it on august 13. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD